In [ ]:
from opentrons import robot, containers, instruments

robot.head_speed(x=18000, y=18000, z=5000, a=700, b=700)


class Vector(object):
    def tolist(self):
        return list(self.input_list)

    def astype(self, input_type):
        if input_type == int:
            return Vector([int(float(x)) for x in self.input_list])
        return Vector([input_type(x) for x in self.input_list])

    def __init__(self, input_list):
        self.input_list = input_list


class DataFrame(object):
    def __len__(self):
        return self.length

    def __getitem__(self, value):
        return Vector(self.dict_input[value])

    def __init__(self, dict_input, length):
        self.dict_input = dict_input
        self.length = length


# Function that reads a csv file correctly without having to import anything. Uses 2 classes, Vector and DataFrame
def read_csv(input_file):
    lines = open(input_file).readlines()
    header = lines[0].rstrip().split(",")
    out_d = {}
    for head in header:
        out_d[head] = []
    for line in lines[1:]:
        spl_line = line.rstrip().split(",")
        for i, head in enumerate(header):
            out_d[head].append(spl_line[i])
    df = DataFrame(out_d, len(lines[1:]))
    return df


# CSV file data
reaction_df = read_csv(r"C:\Users\sdi35357\CODING\github_repo\OT1-coding\Summer_project\csv\p01_r01\reaction_conditions.csv")
reactant_col_df = read_csv(r"C:\Users\sdi35357\CODING\github_repo\OT1-coding\Summer_project\csv\p01_r01\SM_AcOH.csv")
reactant_row_df = read_csv(r"C:\Users\sdi35357\CODING\github_repo\OT1-coding\Summer_project\csv\p01_r01\SM_Amine.csv")
robot.reset()
def reactants_transfer(reactant):
    
    # Deck setup
    tiprack_1000 = containers.load("tiprack-1000ul-H", "B3")
    tiprack_1000_2 = containers.load("tiprack-1000ul-H", "D3")
    source_trough4row = containers.load("trough-12row", "C2")
    rack_stock_reactant_1 = containers.load("FluidX_24_5ml", "A1", "R_1")
    rack_stock_reactant_2 = containers.load("FluidX_24_5ml", "A2", "R_2")
    reaction_rack = containers.load("StarLab_96_tall", "C1")
    trash = containers.load("point", "B3")

    # Pipettes SetUp
    p1000 = instruments.Pipette(
        name='eppendorf1000',
        axis='b',
        trash_container=trash,
        tip_racks=[tiprack_1000, tiprack_1000_2],
        max_volume=1000,
        min_volume=30,
        channels=1,
    )
    
    
    id_header = "Reaction parameters type"
    rack_id = "Rack ID"
    reactant_row = "Reactant row"
    reactant_col = "Reactant col"
    details_header = "Reaction details"
    location_heading = "Location_trough"
    volume_to_add = "Volume to add (uL)"
    number_of_columns = "Number columns"
    number_of_rows = "Number rows"
    rack_1 = "24_rack1"
    rack_2 = "24_rack2"
    location_reactant = "Location"
    
    
    for index, value in enumerate(reaction_df[id_header].tolist()):
        if value == number_of_columns:
            number_cols = int(reaction_df[details_header].tolist()[index])
                
        if value == number_of_rows:
            number_rows = int(reaction_df[details_header].tolist()[index])
                   

        if value == reactant_row:
            volume_to_dispense_row = reaction_df[volume_to_add].tolist()[index]
            if reaction_df[rack_id].tolist()[index] == rack_1:
                rack_stock_row = rack_stock_reactant_1
                rack_stock_col = rack_stock_reactant_2
            else:
                rack_stock_col = rack_stock_reactant_1
                rack_stock_row = rack_stock_reactant_2          
        if value == reactant_col:
            volume_to_dispense_col = reaction_df[volume_to_add].tolist()[index]
            
    for index, location in enumerate (reactant_col_df[location_reactant].tolist()):
        source_location = location
        volume_to_dispense = [volume_to_dispense_col]
        p1000.distribute(volume_to_dispense, rack_stock_col.wells(source_location), [x.top(-15) for x in reaction_rack.rows(index).wells(0, to=number_rows-1)], air_gap=10)
    robot.pause() 

    for index, location in enumerate (reactant_row_df[location_reactant].tolist()):
        source_location = location
        volume_to_dispense = [volume_to_dispense_row]
        p1000.distribute(volume_to_dispense, rack_stock_row.wells(source_location), [x.top(-15) for x in reaction_rack.cols(index).wells(0, to=number_cols-1)], air_gap=10)

    robot.home()

reactants_transfer(reaction_df)
robot.commands()

In [ ]:
        
            
            
        for i, x in enumerate(row_loc_list):
    source_location = x
    volume_to_dispense = [row_vol_list[i]]
    p1000.distribute(volume_to_dispense, source_row.wells(source_location), [x.top(-15) for x in reaction_rack.rows(i).wells(0, to=number_cols)])
        
        for i, x in enumerate(col_loc_list):
    source_location = x
    volume_to_dispense = [col_vol_list[i]]
    p1000.distribute(volume_to_dispense, source_col.wells(source_location), [x.top(-15) for x in reaction_rack.cols(i).wells(0, to=number_rows)])


    location_header = "Location"
    volume_per_reaction_header = "volume_per_vial"
    rack_ID_header = "Rack ID"
    id_header = "CPD ID"
    solvent = "DMA"
    rack_1 = "24_rack1"
    rack_2 = "24_rack2"
    rack_3 = "24_rack3"
    rack_4 = "24_rack4"
    solvent_location_header = "Location_trough"
    solvent_volume_header = "Volume to dispense (uL)"
    counter = 0
    for i, x in enumerate(reactants_df[rack_ID_header].tolist()):
        volume_per_reaction = [reactants_df[volume_per_reaction_header].tolist()[i]]
        source_location = reactants_df[location_header].tolist()[i]

        if x == "":
            #print('null')
            break
        if x == rack_1:
            p1000.pick_up_tip()
            p1000.transfer(volume_per_reaction, rack_stock_reactants_1.wells(source_location),
                           reaction_rack.wells(i + 2).top(-5), new_tip='never')
            p1000.drop_tip()
        if x == rack_2:
            p1000.pick_up_tip()
            p1000.transfer(volume_per_reaction, rack_stock_reactants_2.wells(source_location),
                           reaction_rack.wells(i + 2).top(-5), new_tip='never')
            p1000.drop_tip()
        if x == rack_3:
            p1000.pick_up_tip()
            p1000.transfer(volume_per_reaction, rack_stock_reactants_3.wells(source_location),
                           reaction_rack.wells(i + 2).top(-5), new_tip='never')
            p1000.drop_tip()
        if x == rack_4:
            p1000.pick_up_tip()
            p1000.transfer(volume_per_reaction, rack_stock_reactants_4.wells(source_location),
                           reaction_rack.wells(i + 2).top(-5), new_tip='never')
            p1000.drop_tip()
        counter += 1
    # p1000.pick_up_tip()
    for i, x in enumerate(solvent_df[id_header].tolist()):
        if x == solvent:
            vol_to_add = [solvent_df[solvent_volume_header].tolist()[i]]
            solvent_loc = [solvent_df[solvent_location_header].tolist()[i]]

    p1000.distribute(vol_to_add, source_trough4row.wells(solvent_loc),
                     [x.top() for x in reaction_rack.wells(0, to=counter + 1)])
    #print(counter)
    robot.home()

    
    
    for i, x in enumerate(row_loc_list):
        source_location = x
        volume_to_dispense = [row_vol_list[i]]
        p1000.distribute(volume_to_dispense, source_row.wells(source_location), [x.top(-15) for x in reaction_rack.rows(i).wells(0, to=number_cols)])

    
reactants_transfer(reactants_df, solvent_df)

In [ ]:
robot.head_speed(x=17000,  y=17000,  z=5000, a=700, b=700)
#Deck setup
tiprack_1000 = containers.load("tiprack-1000ul-H", "B3")
source_row = containers.load("FluidX_24_5ml", "A1", "acid")
source_col = containers.load("FluidX_24_5ml", "A2", "amine")
reaction_rack = containers.load("StarLab_96_tall", "D1")
trash = containers.load("point", "C3")

#Pipettes SetUp
p1000 = instruments.Pipette(
    name= 'eppendorf1000',
    axis='b',
    trash_container=trash,
    tip_racks=[tiprack_1000],
    max_volume=1000,
    min_volume=30,
    channels=1,
)
"""
    def do_protocol(self):
        return """
for i, x in enumerate(row_loc_list):
    source_location = x
    volume_to_dispense = [row_vol_list[i]]
    p1000.distribute(volume_to_dispense, source_row.wells(source_location), [x.top(-15) for x in reaction_rack.rows(i).wells(0, to=number_cols)])
for i, x in enumerate(col_loc_list):
    source_location = x
    volume_to_dispense = [col_vol_list[i]]
    p1000.distribute(volume_to_dispense, source_col.wells(source_location), [x.top(-15) for x in reaction_rack.cols(i).wells(0, to=number_rows)])
robot.home()
        """



In [26]:
"""Protocol that uses the multichannel to add base and coupling agent to the desired number of rows in the plate."""

from opentrons import robot, containers, instruments
robot.head_speed(x=16000, y=16000, z=3000, a=400, b=400)

class Vector(object):
    def tolist(self):
        return list(self.input_list)

    def astype(self, input_type):
        if input_type == int:
            return Vector([int(float(x)) for x in self.input_list])
        return Vector([input_type(x) for x in self.input_list])

    def __init__(self, input_list):
        self.input_list = input_list

class DataFrame(object):
    def __len__(self):
        return self.length

    def __getitem__(self, value):
        return Vector(self.dict_input[value])

    def __init__(self, dict_input, length):
        self.dict_input = dict_input
        self.length = length
#Function that reads a csv file correctly without having to import anything (issues with molport). Uses 2 classes, Vector and DataFrame
def read_csv(input_file):
    lines = open(input_file).readlines()
    header = lines[0].rstrip().split(",")
    out_d = {}
    for head in header:
        out_d[head] = []
    for line in lines[1:]:
        spl_line = line.rstrip().split(",")
        for i, head in enumerate(header):
            out_d[head].append(spl_line[i])
    df = DataFrame(out_d, len(lines[1:]))
    return df
robot.reset()
#CSV file data

reaction_df = read_csv(r"C:\Users\sdi35357\CODING\github_repo\OT1-coding\Summer_project\csv\p01_r01\reaction_conditions.csv")
#reactant_col_df = read_csv(
#    r"C:\Users\sdi35357\CODING\github_repo\OT1-coding\Summer_project\csv\p01_r01\SM_Amine.csv")
#reactant_row_df = read_csv(
#    r"C:\Users\sdi35357\CODING\github_repo\OT1-coding\Summer_project\csv\p01_r01\SM_AcOH.csv")


# Deck setup
tiprack_300 = containers.load("tiprack-300ul", "D3")
source_trough12row = containers.load('trough-12row', "E2")
reaction_rack = containers.load("StarLab_96_tall", "D1")
trash = containers.load("point", "B3")

# Pipettes SetUp
p300_multi = instruments.Pipette(
    name='dlab_300multi',
    axis="a",
    trash_container=trash,
    tip_racks=[tiprack_300],
    max_volume=300,
    min_volume=30,
    channels=8,
)

def add_base_and_couplingAgent(reaction_condition):
    id_header = "Reaction parameters type"
    details_header = "Reaction details"
    base = "Reagent 1"
    coupling = "Reagent 2"
    location_heading = "Location_trough"
    volume_to_add = "Volume to add (uL)"
    number_of_columns = "Number columns"
    
    
    for index, value in enumerate(reaction_df[id_header].tolist()):
        if value == base:
            base_location = reaction_df[location_heading].tolist()[index]
            base_volume = float(reaction_df[volume_to_add].tolist()[index])
        if value == coupling:
            coupling_location = reaction_df[location_heading].tolist()[index]
            coupling_volume = float(reaction_df[volume_to_add].tolist()[index])
        if value == number_of_columns:
            #number_cols = float(reaction_df[details_header].tolist()[index])
            number_cols = reaction_df[details_header].tolist()[index]

            

    print(base_location, type(base_location), base_volume, type(base_volume), number_cols, type(number_cols))


    p300_multi.distribute(base_volume, source_trough12row.wells(base_location),
                                  [x.top() for x in reaction_rack.rows(0, to=number_cols)], air_gap=10)
    p300_multi.distribute(coupling_volume, source_trough12row.wells(coupling_location),
                                  [x.top() for x in reaction_rack.rows(0, to=number_cols)], air_gap=10)
    robot.home()

add_base_coupling(reaction_df)

robot.commands()

A1 <class 'str'> 30.48113208 <class 'float'> 12 <class 'str'>


['Picking up tip from <WellSeries: <Well A1><Well B1><Well C1><Well D1><Well E1><Well F1><Well G1><Well H1>>',
 'Aspirating 273.84905664 at <Deck><Slot E2><Container trough-12row><Well A1>',
 'Air gap',
 'Aspirating 10 ',
 'Dispensing 10 at <WellSeries: <Well A1><Well B1><Well C1><Well D1><Well E1><Well F1><Well G1><Well H1>>',
 'Dispensing 30.48113208 at <WellSeries: <Well A1><Well B1><Well C1><Well D1><Well E1><Well F1><Well G1><Well H1>>',
 'Air gap',
 'Aspirating 10 ',
 'Dispensing 10 at <WellSeries: <Well A2><Well B2><Well C2><Well D2><Well E2><Well F2><Well G2><Well H2>>',
 'Dispensing 30.48113208 at <WellSeries: <Well A2><Well B2><Well C2><Well D2><Well E2><Well F2><Well G2><Well H2>>',
 'Air gap',
 'Aspirating 10 ',
 'Dispensing 10 at <WellSeries: <Well A3><Well B3><Well C3><Well D3><Well E3><Well F3><Well G3><Well H3>>',
 'Dispensing 30.48113208 at <WellSeries: <Well A3><Well B3><Well C3><Well D3><Well E3><Well F3><Well G3><Well H3>>',
 'Air gap',
 'Aspirating 10 ',
 'Dispensi

In [19]:
from opentrons import robot, containers, instruments

robot.head_speed(x=18000, y=18000, z=5000, a=700, b=700)


class Vector(object):
    def tolist(self):
        return list(self.input_list)

    def astype(self, input_type):
        if input_type == int:
            return Vector([int(float(x)) for x in self.input_list])
        return Vector([input_type(x) for x in self.input_list])

    def __init__(self, input_list):
        self.input_list = input_list


class DataFrame(object):
    def __len__(self):
        return self.length

    def __getitem__(self, value):
        return Vector(self.dict_input[value])

    def __init__(self, dict_input, length):
        self.dict_input = dict_input
        self.length = length


# Function that reads a csv file correctly without having to import anything (issues with molport). Uses 2 classes, Vector and DataFrame
def read_csv(input_file):
    lines = open(input_file).readlines()
    header = lines[0].rstrip().split(",")
    out_d = {}
    for head in header:
        out_d[head] = []
    for line in lines[1:]:
        spl_line = line.rstrip().split(",")
        for i, head in enumerate(header):
            out_d[head].append(spl_line[i])
    df = DataFrame(out_d, len(lines[1:]))
    return df


# CSV file data
solvent_df = read_csv(r"C:\Users\sdi35357\CODING\github_repo\OT1-coding\Summer_project\csv\p01_r01\reaction_conditions.csv")
reactant_col_df = read_csv(
    r"C:\Users\sdi35357\CODING\github_repo\OT1-coding\Summer_project\csv\p01_r01\SM_Amine.csv")
reactant_row_df = read_csv(
    r"C:\Users\sdi35357\CODING\github_repo\OT1-coding\Summer_project\csv\p01_r01\SM_AcOH.csv")

def stock_solution_reactant(reactants_df, solvent_df):
    # Deck setup
    tiprack_1000 = containers.load("tiprack-1000ul-H", "B3")
    tiprack_1000_2 = containers.load("tiprack-1000ul-H", "D3")
    source_trough4row = containers.load("trough-12row", "C2")
    rack_stock_reactants_1 = containers.load("FluidX_24_5ml", "A1", "R_1")
    rack_stock_reactants_2 = containers.load("FluidX_24_5ml", "A2", "R_2")
    rack_stock_reactants_3 = containers.load("FluidX_24_5ml", "B1", "R_3")
    rack_stock_reactants_4 = containers.load("FluidX_24_5ml", "B2", "R_4")
    trash = containers.load("point", "C3")

    # Pipettes SetUp
    p1000 = instruments.Pipette(
        name='eppendorf1000',
        axis='b',
        trash_container=trash,
        tip_racks=[tiprack_1000, tiprack_1000_2],
        max_volume=1000,
        min_volume=30,
        channels=1,
    )
    rack_ID_header = "Rack ID"
    id_header = "Reaction parameters type"
    solvent = "Reaction solvent"
    rack_1 = "24_rack1"
    rack_2 = "24_rack2"
    rack_3 = "24_rack3"
    rack_4 = "24_rack4"
    location_header = "Location_trough"
    destination_location_header = "Location"
    volume_stock_header = "Volume to dispense (exp) at 0.8M"

    for i, x in enumerate(solvent_df[id_header].tolist()):
        if x == solvent:
            solvent_location = solvent_df[location_header].tolist()[i]

    p1000.pick_up_tip()

    for i, x in enumerate(reactants_df[destination_location_header].tolist()):
        destination_location = x
        vol_to_dispense = [reactants_df[volume_stock_header].tolist()[i]]
        reactants_id = reactants_df[rack_ID_header].tolist()[i]
        if reactants_id == "":
            print ('null')
            break
        # print (rack_ID_header, reactants_id, vol_to_dispense)
        if reactants_id == rack_1:
            print ('rack1')
            if vol_to_dispense != 0:
                p1000.transfer(vol_to_dispense, source_trough4row.wells(solvent_location),
                               rack_stock_reactants_1.wells(destination_location).top(-5), new_tip='never', air_gap=10)
        if reactants_id == rack_2:
            print ('rack2')
            if vol_to_dispense != 0:
                p1000.transfer(vol_to_dispense, source_trough4row.wells(solvent_location),
                               rack_stock_reactants_2.wells(destination_location).top(-5), new_tip='never', air_gap=10)
        if reactants_id == rack_3:
            print ('rack3')
            if vol_to_dispense != 0:
                p1000.transfer(vol_to_dispense, source_trough4row.wells(solvent_location),
                               rack_stock_reactants_3.wells(destination_location).top(-5), new_tip='never', air_gap=10)
        if reactants_id == rack_4:
            print ('rack4')
            if vol_to_dispense != 0:
                p1000.transfer(vol_to_dispense, source_trough4row.wells(solvent_location),
                               rack_stock_reactants_4.wells(destination_location).top(-5), new_tip='never', air_gap=10)
    p1000.drop_tip()

    robot.home()

stock_solution_reactant(reactant_col_df, solvent_df)
stock_solution_reactant(reactant_row_df, solvent_df)

rack2
rack2
rack2
rack2
rack2
rack2
rack2
rack2
rack1
rack1
rack1
rack1
rack1
rack1
rack1
rack1
rack1
rack1


In [18]:
from opentrons import robot, containers, instruments

robot.head_speed(x=18000, y=18000, z=5000, a=700, b=700)

class Vector(object):
    def tolist(self):
        return list(self.input_list)

    def astype(self, input_type):
        if input_type == int:
            return Vector([int(float(x)) for x in self.input_list])
        return Vector([input_type(x) for x in self.input_list])

    def __init__(self, input_list):
        self.input_list = input_list


class DataFrame(object):
    def __len__(self):
        return self.length

    def __getitem__(self, value):
        return Vector(self.dict_input[value])

    def __init__(self, dict_input, length):
        self.dict_input = dict_input
        self.length = length


# Function that reads a csv file correctly without having to import anything (issues with molport). Uses 2 classes, Vector and DataFrame
def read_csv(input_file):
    lines = open(input_file).readlines()
    header = lines[0].rstrip().split(",")
    out_d = {}
    for head in header:
        out_d[head] = []
    for line in lines[1:]:
        spl_line = line.rstrip().split(",")
        for i, head in enumerate(header):
            out_d[head].append(spl_line[i])
    df = DataFrame(out_d, len(lines[1:]))
    return df


# CSV file data
solvent_df = read_csv(r"C:\Users\sdi35357\CODING\github_repo\OT1-coding\Summer_project\csv\p01_r01\solvents.csv")
reactant_col_df = read_csv(
    r"C:\Users\sdi35357\CODING\github_repo\OT1-coding\Summer_project\csv\p01_r01\SM_Amine.csv")
reactant_row_df = read_csv(
    r"C:\Users\sdi35357\CODING\github_repo\OT1-coding\Summer_project\csv\p01_r01\SM_AcOH.csv")
robot.reset()
def transfer_24racks(reactants_df, solvent_df):
     # Deck setup
    tiprack_1000 = containers.load("tiprack-1000ul-H", "B3")
    tiprack_1000_2 = containers.load("tiprack-1000ul-H", "D3")
    source_trough4row = containers.load("trough-12row", "C2")
    rack_stock_reactants_1 = containers.load("FluidX_24_5ml", "A1", "R_1")
    rack_stock_reactants_2 = containers.load("FluidX_24_5ml", "A2", "R_2")
    rack_stock_reactants_3 = containers.load("FluidX_24_5ml", "B1", "R_3")
    rack_stock_reactants_4 = containers.load("FluidX_24_5ml", "B2", "R_4")
    trash = containers.load("point", "C3")
    
    # Pipettes SetUp
    p1000 = instruments.Pipette(
        name='eppendorf1000',
        axis='b',
        trash_container=trash,
        tip_racks=[tiprack_1000,tiprack_1000_2],
        max_volume=1000,
        min_volume=30,
        channels=1,
    )
    rack_ID_header = "Rack ID"
    id_header = "CPD ID"
    solvent = "DMA"
    rack_1 = "24_rack1"
    rack_2 = "24_rack2"
    rack_3 = "24_rack3"
    rack_4 = "24_rack4"
    location_header = "Location_trough"
    destination_location_header = "Location"
    volume_stock_header = "Volume to dispense (exp) at 0.8M"
    
    for i, x in enumerate(solvent_df[id_header].tolist()):
        if x == solvent:
            solvent_location = solvent_df[location_header].tolist()[i]

    
    p1000.pick_up_tip()
    
    for i, x in enumerate(reactants_df[destination_location_header].tolist()):
        destination_location = x
        vol_to_dispense = [reactants_df[volume_stock_header].tolist()[i]]
        reactants_id = reactants_df[rack_ID_header].tolist()[i]
        if reactants_id == "":
            print ('null')
            break
        #print (rack_ID_header, reactants_id, vol_to_dispense)
        if reactants_id == rack_1:
            print ('rack1')
            if vol_to_dispense != 0:
                p1000.transfer(vol_to_dispense, source_trough4row.wells(solvent_location),
                               rack_stock_reactants_1.wells(destination_location).top(-5), new_tip='never', air_gap = 10)
        if reactants_id == rack_2:
            print ('rack2')
            if vol_to_dispense != 0:
                p1000.transfer(vol_to_dispense, source_trough4row.wells(solvent_location),
                               rack_stock_reactants_2.wells(destination_location).top(-5), new_tip='never',air_gap = 10)
        if reactants_id == rack_3:
            print ('rack3')
            if vol_to_dispense != 0:
                p1000.transfer(vol_to_dispense, source_trough4row.wells(solvent_location),
                               rack_stock_reactants_3.wells(destination_location).top(-5), new_tip='never',air_gap = 10)
        if reactants_id == rack_4:
            print ('rack4')
            if vol_to_dispense != 0:
                p1000.transfer(vol_to_dispense, source_trough4row.wells(solvent_location),
                               rack_stock_reactants_4.wells(destination_location).top(-5), new_tip='never',air_gap = 10)
    p1000.drop_tip()
    
    robot.home()
    
transfer_24racks(reactant_col_df,solvent_df)
transfer_24racks(reactant_row_df,solvent_df)
robot.commands()


rack2
rack2
rack2
rack2
rack2
rack2
rack2
rack2
rack1
rack1
rack1
rack1
rack1
rack1
rack1
rack1
rack1
rack1


['Picking up tip from <Deck><Slot B3><Container tiprack-1000ul-H><Well A1>',
 'Aspirating 990.0 at <Deck><Slot C2><Container trough-12row><Well A8>',
 'Air gap',
 'Aspirating 10 ',
 'Dispensing 10 at <Deck><Slot A2><Container R_2><Well C1>',
 'Dispensing 990.0 at <Deck><Slot A2><Container R_2><Well C1>',
 'Aspirating 659.433978 at <Deck><Slot C2><Container trough-12row><Well A8>',
 'Air gap',
 'Aspirating 10 ',
 'Dispensing 10 at <Deck><Slot A2><Container R_2><Well C1>',
 'Dispensing 659.433978 at <Deck><Slot A2><Container R_2><Well C1>',
 'Aspirating 659.433978 at <Deck><Slot C2><Container trough-12row><Well A8>',
 'Air gap',
 'Aspirating 10 ',
 'Dispensing 10 at <Deck><Slot A2><Container R_2><Well C1>',
 'Dispensing 659.433978 at <Deck><Slot A2><Container R_2><Well C1>',
 'Aspirating 979.8018785 at <Deck><Slot C2><Container trough-12row><Well A8>',
 'Air gap',
 'Aspirating 10 ',
 'Dispensing 10 at <Deck><Slot A2><Container R_2><Well C2>',
 'Dispensing 979.8018785 at <Deck><Slot A2><C

In [6]:
def stock_solution_reactants(reactant_col_df, reactant_row_df, solvent_df):
    # Deck setup
    tiprack_1000 = containers.load("tiprack-1000ul-H", "B3")
    tiprack_1000_2 = containers.load("tiprack-1000ul-H", "D3")
    source_trough4row = containers.load("trough-12row", "C2")
    rack_stock_reactants_1 = containers.load("FluidX_24_5ml", "A1", "R_1")
    rack_stock_reactants_2 = containers.load("FluidX_24_5ml", "A2", "R_2")
    rack_stock_reactants_3 = containers.load("FluidX_24_5ml", "B1", "R_3")
    rack_stock_reactants_4 = containers.load("FluidX_24_5ml", "B2", "R_4")
    trash = containers.load("point", "C3")
    
    # Pipettes SetUp
    p1000 = instruments.Pipette(
        name='eppendorf1000',
        axis='b',
        trash_container=trash,
        tip_racks=[tiprack_1000,tiprack_1000_2],
        max_volume=1000,
        min_volume=30,
        channels=1,
    )
    rack_ID_header = "Rack ID"
    id_header = "CPD ID"
    solvent = "DMA"
    rack_1 = "24_rack1"
    rack_2 = "24_rack2"
    rack_3 = "24_rack3"
    rack_4 = "24_rack4"
    location_header = "Location_trough"
    destination_location_header = "Location"
    volume_stock_header = "Volume to dispense (exp) at 0.8M"
    
    

    for i, x in enumerate(solvent_df[id_header].tolist()):
        if x == solvent:
            solvent_location = solvent_df[location_header].tolist()[i]
    
    transfer_24racks(reactant_col_df, destination_location_header, volume_stock_header,rack_ID_header)
    transfer_24racks(reactant_row_df, destination_location_header, volume_stock_header,rack_ID_header)
    robot.home()


stock_solution_reactants(reactant_col_df, reactant_row_df, solvent_df)

NameError: name 'reactants_df' is not defined

In [ ]:
'''OT1 -Single channel- Making up of main reagent stock solution (2 maximum) using a defined solvent'''

from opentrons import robot, containers, instruments

robot.head_speed(x=21000, y=21000, z=5000, a=700, b=700)


class Vector(object):
    def tolist(self):
        return list(self.input_list)

    def astype(self, input_type):
        if input_type == int:
            return Vector([int(float(x)) for x in self.input_list])
        return Vector([input_type(x) for x in self.input_list])

    def __init__(self, input_list):
        self.input_list = input_list


class DataFrame(object):
    def __len__(self):
        return self.length

    def __getitem__(self, value):
        return Vector(self.dict_input[value])

    def __init__(self, dict_input, length):
        self.dict_input = dict_input
        self.length = length


# Function that reads a csv file correctly without having to import anything (issues with molport). Uses 2 classes, Vector and DataFrame
def read_csv(input_file):
    lines = open(input_file).readlines()
    header = lines[0].rstrip().split(",")
    out_d = {}
    for head in header:
        out_d[head] = []
    for line in lines[1:]:
        spl_line = line.rstrip().split(",")
        for i, head in enumerate(header):
            out_d[head].append(spl_line[i])
    df = DataFrame(out_d, len(lines[1:]))
    return df


# CSV file data
reaction_col_df = read_csv(
    r"C:\Users\sdi35357\CODING\github_repo\OT1-coding\Summer_project\csv\p01_r01\SM_Amine.csv")
reactant_row_df = read_csv(
    r"C:\Users\sdi35357\CODING\github_repo\OT1-coding\Summer_project\csv\p01_r01\SM_AcOH.csv")


"""Function that does one liquid handling transfers. Dilute a solid reagent in a big trough to the right concentration.
It requires 2 csv files, the custom made stock reagent csv."""

def stock_solution_main(reaction_conditions, reactants):
    # Deck setup
    tiprack_1000 = containers.load("tiprack-1000ul-H", "B3")
    source_trough4row = containers.load("trough-12row", "C2")
    trash = containers.load("point", "C3")
    # Pipettes SetUp
    p1000 = instruments.Pipette(
        name='eppendorf1000',
        axis='b',
        trash_container=trash,
        tip_racks=[tiprack_1000],
        max_volume=1000,
        min_volume=30,
        channels=1,
    )

    id_header = "reaction"
    reaction_to_start = "Coupling_sequence"
    solvent_trough_location_header = "main reactant solvent Location"
    volume_stock_header = "main reactant volume to add - stock (uL)"
    destination_header = "main reactant location"
    additional_base_header = "Additional solvent to add (uL)"
    additional_base_location_header = "additional solvent location"

    for index, value in enumerate(reaction_conditions_df[id_header].tolist()):
        if value == reaction_to_start:
            solvent_location = reaction_conditions_df[solvent_trough_location_header].tolist()[index]
            solvent_volume = reaction_conditions_df[volume_stock_header].tolist()[index]
            destination_location = reaction_conditions_df[destination_header].tolist()[index]
            base_to_add = reaction_conditions_df[additional_base_header].tolist()[index]
            base_location = reaction_conditions_df[additional_base_location_header].tolist()[index]

    p1000.pick_up_tip()
    p1000.transfer([solvent_volume], source_trough4row.wells(solvent_location),
                   source_trough4row.wells(destination_location).top(-5), new_tip='never')
    p1000.drop_tip()
    p1000.pick_up_tip()
    p1000.transfer([base_to_add], source_trough4row.wells(base_location),
                   source_trough4row.wells(destination_location).top(-5), new_tip='never')
    p1000.drop_tip()
    robot.home()


stock_solution_main(reaction_conditions_df, reactant_df)


In [ ]:
id_header = process["id_header"]
        solvent = process["solvent"]
        location_header = process["location_header"]
        volume_stock_header = process["volume_stock_header"]

        # CSV file data
        row_csv = input_dict["files"]["row_csv"]
        col_csv = input_dict["files"]["col_csv"]
        trough_csv = input_dict["files"]["trough_csv"]

        # Now define the lists
        self.list_vars = {
            "row_vol_list": {"file": row_csv, "header": volume_stock_header},
            "row_loc_list": {"file": row_csv, "header": location_header},
            "col_vol_list": {"file": col_csv, "header": volume_stock_header},
            "col_loc_list": {"file": col_csv, "header": location_header},
        }
        self.trough_vars = {
            "path": trough_csv,
            "id_header": id_header,
            "solvent_location": {"col_header": location_header, "solvent_name": solvent}
        }

    def do_setup(self):
        return """
robot.head_speed(x=18000,  y=18000,  z=5000, a=700, b=700)
#Deck setup
tiprack_1000 = containers.load("tiprack-1000ul-H", "B3")
source_trough4row = containers.load("trough-12row", "C2")
destination_row = containers.load("FluidX_24_5ml", "A1", "acid")
destination_col = containers.load("FluidX_24_5ml", "A2", "amine")
trash = containers.load("point", "C3")
#Pipettes SetUp
p1000 = instruments.Pipette(
    name= 'eppendorf1000',
    axis='b',
    trash_container=trash,
    tip_racks=[tiprack_1000],
    max_volume=1000,
    min_volume=30,
    channels=1,
)
"""

    def do_protocol(self):
        return """
# Now define the actions
p1000.pick_up_tip()
for i, destination_location in enumerate(row_loc_list):
    vol_to_dispense = [row_vol_list[i]]
    if vol_to_dispense != 0:
        p1000.transfer(vol_to_dispense, source_trough4row.wells(solvent_location), destination_row.wells(destination_location).top(-5), new_tip = 'never')
for i, destination_location in enumerate(col_loc_list):
    vol_to_dispense = [col_vol_list[i]]
    if vol_to_dispense != 0:
        p1000.transfer(vol_to_dispense, source_trough4row.wells(solvent_location), destination_col.wells(destination_location).top(-5), new_tip = 'never')
p1000.drop_tip()
robot.home()""